<a href="https://colab.research.google.com/github/vincent-grari/DIVERS/blob/main/Clara.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [103]:
from matplotlib import pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import sys
print(sys.argv)
import os
import urllib
import os.path
import sklearn.preprocessing as preprocessing
from collections import namedtuple
import pandas as pd

['/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py', '-f', '/root/.local/share/jupyter/runtime/kernel-ec1d182b-e3b1-4021-8285-99a785811de8.json']


In [104]:
pip install --upgrade scikit-learn

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.7/dist-packages (0.24.2)


In [105]:
!wget "http://grarivincent.com/research/baseINSEE.csv"

--2021-06-29 14:24:48--  http://grarivincent.com/research/baseINSEE.csv
Resolving grarivincent.com (grarivincent.com)... 217.160.0.183, 2001:8d8:100f:f000::2ce
Connecting to grarivincent.com (grarivincent.com)|217.160.0.183|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5107089 (4.9M) [text/csv]
Saving to: ‘baseINSEE.csv.3’

baseINSEE.csv.3     100%[===================>]   4.87M  3.77MB/s    in 1.3s    

2021-06-29 14:24:49 (3.77 MB/s) - ‘baseINSEE.csv.3’ saved [5107089/5107089]



In [106]:
baseINSEE = (pd.read_csv("baseINSEE.csv", #names=column_names,
                    sep=r'\s*;\s*', engine='python', na_values=['NA']))

In [107]:
baseINSEE['pol_insee_code']= baseINSEE['code_commune_INSEE']

In [108]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('../..')))
def load_ICU_Pricing(path):
    #column_names = ["CalYear","Gender","Type","Category","Occupation","Age","Group1","Bonus","Poldur","Value","Adind","SubGroup2","Group2","Density","Y"]
    #input_data = (pd.read_csv(path, #names=column_names,
    #                           sep=r',', engine='python', keep_default_na=False, na_values=['NA']))
    
    path = "http://grarivincent.com/research/db4ModelFull2.csv"
    input_data = (pd.read_csv(path, #names=column_names,
                                sep=r',', engine='python', na_values=['NA']))
    input_data=input_data.drop(columns=['Unnamed: 0'])
    
    base=baseINSEE
    base = base[~base.pol_insee_code.str[:2].isin(["2A","2B"])]
    base["pol_insee_code"] = base["pol_insee_code"].astype(int)
    base_nd =  base.drop_duplicates(subset=['pol_insee_code'])

    input_data = input_data.drop_duplicates()
    input_data = input_data[~input_data.pol_insee_code.str[:2].isin(["2A","2B"])]
    input_data["pol_insee_code"] = input_data["pol_insee_code"].astype(int)

    input_data = input_data.merge(base_nd, how='inner', on="pol_insee_code")
    #input_data = input_data[input_data['claim_amount']<=5000]
    #input_data=input_data[input_data['claim_amount']!=0]
   
    #
    sensitive_attribs = ['Age']
    print(input_data.shape)
    G0 = (input_data.iloc[:, 37:180]) #137])
    #Z0 = pd.concat(Z0,input_data['long'])
    G0=G0.select_dtypes([np.number])
    G0 = G0.fillna(G0.mean())
    G=G0
    #Z = (Z0
    #    .drop(columns=['"REG"'
    #                   ])
    #    .fillna('Unknown').pipe(pd.get_dummies))

    y = input_data['claim_amount']
    E  = input_data['exposure']
    #I = input_data["pol_insee_code"]
    I = input_data[['pol_insee_code','latitude','longitude']]

    # features; note that the 'target' and sentive attribute columns are dropped
    X0 = input_data.iloc[:, 4:32]
    
    X = (X0
          .drop(columns=['pol_coverage','pol_insee_code'])
          .fillna('Unknown').pipe(pd.get_dummies))
    INSEE = input_data.iloc[:, 36]
    print(f"features X: {X.shape[0]} samples, {X.shape[1]} attributes")
    print(f"targets y: {y.shape[0]} samples")
    print(f"sensitives E: {E.shape[0]} samples, {E.shape[0]} attributes")
    return X, y, G, E, I

X, y, G, E, I =load_ICU_Pricing("http://grarivincent.com/research/db4modelfull3.csv")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


(98417, 151)
features X: 98417 samples, 1168 attributes
targets y: 98417 samples
sensitives E: 98417 samples, 98417 attributes


In [109]:
X

,pol_bonus,pol_duration,pol_sit_duration,drv_age1,drv_age2,drv_age_lic1,drv_age_lic2,vh_age,vh_cyl,vh_din,vh_sale_begin,vh_sale_end,vh_speed,vh_value,vh_weight,pol_pay_freq_Biannual,pol_pay_freq_Monthly,pol_pay_freq_Quarterly,pol_pay_freq_Yearly,pol_payd_No,pol_payd_Yes,pol_usage_Professional,pol_usage_Retired,pol_usage_WorkPrivate,drv_drv2_No,drv_drv2_Yes,drv_sex1_F,drv_sex1_M,drv_sex2_F,drv_sex2_M,drv_sex2_Unknown,vh_fuel_Diesel,vh_fuel_Gasoline,vh_fuel_Hybrid,vh_make_ACL,vh_make_ALFA ROMEO,vh_make_ALPINE,vh_make_APAL,vh_make_ARO,vh_make_ASIA,...,vh_model_XG,vh_model_XJ,vh_model_XJ6,vh_model_XJ8,vh_model_XJS,vh_model_XK,vh_model_XK150,vh_model_XK8,vh_model_XKR,vh_model_XM,vh_model_XSARA,vh_model_Y,vh_model_Y10,vh_model_YARIS,vh_model_YETI,vh_model_YPSILON,vh_model_YRV,vh_model_Z3,vh_model_Z4,vh_model_ZAFIRA,vh_model_ZETA,vh_model_ZR,vh_model_ZS,vh_model_ZT,vh_model_ZT-T,vh_model_ZX,vh_model_ZX COUPE,vh_type_Commercial,vh_type_Tourism,vh_make_new_CITROEN,vh_make_new_FIAT,vh_make_new_FORD,vh_make_new_MERCEDES BENZ,vh_make_new_NISSAN,vh_make_new_OPEL,vh_make_new_Other,vh_make_new_PEUGEOT,vh_make_new_RENAULT,vh_make_new_TOYOTA,vh_make_new_VOLKSWAGEN
0,0.5,29,9,85,0,62,0,10,1587,98,10,9,182,20700,1210,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
1,0.5,12,3,57,57,32,38,8,1868,70,9,6,142,14400,1185,0,0,0,1,1,0,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
2,0.5,3,1,69,0,39,0,4,2149,170,4,2,229,34250,1510,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
3,0.5,2,2,40,38,22,17,6,2198,155,6,1,215,39350,1575,0,0,0,1,1,0,0,0,1,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,0.5,13,1,69,0,45,0,5,1984,150,6,4,213,25500,1389,0,0,0,1,1,0,0,0,1,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98412,0.5,1,1,61,0,42,0,6,2476,140,8,5,165,24853,1922,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
98413,0.5,5,5,43,0,24,0,17,1769,60,28,18,155,11564,850,0,1,0,0,1,0,0,0,1,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
98414,0.5,6,6,71,0,48,0,23,770,34,25,18,125,6472,700,0,0,0,1,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
98415,0.5,7,2,60,0,32,0,7,1868,70,13,8,138,17342,0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0


In [110]:
G.shape

(98417, 90)

In [111]:
(y!=0).sum()

11025

In [112]:
X=X[y>0]
G=G[y>0]
E=E[y>0]
I=I[y>0]
y=y[y>0]

In [113]:
y.mean()

1107.7378757369586

In [114]:
X.shape

(11025, 1168)

In [115]:
(y>0).mean()

1.0

In [116]:
y.mean()

1107.7378757369586

In [117]:
G= G.iloc[:, :-5]

In [118]:
X_train, X_test, y_train, y_test, G_train, G_test, E_train, E_test, I_train, I_test  = train_test_split(X, y, G, E, I, test_size=0.2, random_state=7)

In [119]:
scaler = StandardScaler().fit(X_train)
scale_df = lambda df, scaler: pd.DataFrame(scaler.transform(df), columns=df.columns, index=df.index)
X_train = X_train.pipe(scale_df, scaler) 
X_test = X_test.pipe(scale_df, scaler)

In [120]:
scalerg = StandardScaler().fit(G_train)
scale_df_g = lambda df, scaler: pd.DataFrame(scalerg.transform(df), columns=df.columns, index=df.index)
G_train = G_train.pipe(scale_df_g, scalerg) 
G_test = G_test.pipe(scale_df_g, scalerg)

In [121]:
y_traint =np.expand_dims(y_train,axis=1)
y_testt =np.expand_dims(y_test,axis=1)

In [122]:
from sklearn import linear_model

### **GLM POISSON SANS ZONIER **

In [123]:
from sklearn import linear_model
reg = linear_model.GammaRegressor() 
reg.fit(X_train,y_train)

GammaRegressor()

In [124]:
def EDR_GAMMA(yhat, y):
  #loss=torch.mean(torch.exp(xbeta)-y*xbeta)
  #loss=torch.mean(yhat-y*torch.log(yhat))
  eps=0.000000000001
  res=1-np.mean(-np.log(y/yhat)+(y-yhat)/yhat)/np.mean(-np.log(y/np.mean(y))+(y-np.mean(y))/np.mean(y))
  return res

def gini_coefficient(x):
    """Compute Gini coefficient of array of values"""
    diffsum = 0
    for i, xi in enumerate(x[:-1], 1):
        diffsum += np.sum(np.abs(xi - x[i:]))
    return diffsum / (len(x)**2 * np.mean(x))

In [125]:
print(" GINI train : ", gini_coefficient(reg.predict(X_train)))
print(" GINI test : ", gini_coefficient(reg.predict(X_test)))
print(" EDR POIS train : ", EDR_GAMMA(reg.predict(X_train),y_train))
print(" EDR POIS test : ", EDR_GAMMA(reg.predict(X_test),y_test))
MSE_Poi_tr =  ((reg.predict(X_train)-y_train)**2).mean()
MSE_Poi_test =  ((reg.predict(X_test)-y_test)**2).mean()
print(" MSE Poi train : ", MSE_Poi_tr)
print(" MSE Poi test : ", MSE_Poi_test)

 GINI train :  0.10070031409441398
 GINI test :  0.09942786904759496
 EDR POIS train :  0.07213321146505158
 EDR POIS test :  0.007717492240731261
 MSE Poi train :  3232115.926246228
 MSE Poi test :  19714604.386624306


In [127]:
reg.predict(X_train)

array([1194.93135431, 1037.87358184, 1014.1938358 , ...,  991.20866891,
       1057.77039815, 1086.86686048])

In [128]:
reg.score(X_train,y_train)

0.07213321146504448

In [129]:
X_train.shape

(8820, 1168)

In [131]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('../..')))
def load_ICU_Pricing(path):
    #column_names = ["CalYear","Gender","Type","Category","Occupation","Age","Group1","Bonus","Poldur","Value","Adind","SubGroup2","Group2","Density","Y"]
    #input_data = (pd.read_csv(path, #names=column_names,
    #                           sep=r',', engine='python', keep_default_na=False, na_values=['NA']))
    
    path = "http://grarivincent.com/research/db4ModelFull2.csv"
    input_data = (pd.read_csv(path, #names=column_names,
                                sep=r',', engine='python', na_values=['NA']))
    input_data=input_data.drop(columns=['Unnamed: 0'])
    
    base=baseINSEE
    base = base[~base.pol_insee_code.str[:2].isin(["2A","2B"])]
    base["pol_insee_code"] = base["pol_insee_code"].astype(int)
    base_nd =  base.drop_duplicates(subset=['pol_insee_code'])

    input_data = input_data.drop_duplicates()
    input_data = input_data[~input_data.pol_insee_code.str[:2].isin(["2A","2B"])]
    input_data["pol_insee_code"] = input_data["pol_insee_code"].astype(int)

    input_data = input_data.merge(base_nd, how='inner', on="pol_insee_code")
    #input_data = input_data[input_data['claim_amount']<=5000]
    #input_data=input_data[input_data['claim_amount']!=0]
   
    #
    sensitive_attribs = ['Age']
    print(input_data.shape)
    G0 = (input_data.iloc[:, 37:180]) #137])
    #Z0 = pd.concat(Z0,input_data['long'])
    G0=G0.select_dtypes([np.number])
    G0 = G0.fillna(G0.mean())
    G=G0
    #Z = (Z0
    #    .drop(columns=['"REG"'
    #                   ])
    #    .fillna('Unknown').pipe(pd.get_dummies))

    y = input_data['claim_nb']
    E  = input_data['exposure']
    #I = input_data["pol_insee_code"]
    I = input_data[['pol_insee_code','latitude','longitude']]

    # features; note that the 'target' and sentive attribute columns are dropped
    X0 = input_data.iloc[:, 4:32]
    
    X = (X0
          .drop(columns=['pol_coverage','pol_insee_code'])
          .fillna('Unknown').pipe(pd.get_dummies))
    INSEE = input_data.iloc[:, 36]
    print(f"features X: {X.shape[0]} samples, {X.shape[1]} attributes")
    print(f"targets y: {y.shape[0]} samples")
    print(f"sensitives E: {E.shape[0]} samples, {E.shape[0]} attributes")
    return X, y, G, E, I

X, y, G, E, I =load_ICU_Pricing("http://grarivincent.com/research/db4modelfull3.csv")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


(98417, 151)
features X: 98417 samples, 1168 attributes
targets y: 98417 samples
sensitives E: 98417 samples, 98417 attributes


In [133]:
path = "http://grarivincent.com/research/db4ModelFull2.csv"
input_data = (pd.read_csv(path, #names=column_names,
                            sep=r',', engine='python', na_values=['NA']))
input_data=input_data.drop(columns=['Unnamed: 0'])

base=baseINSEE
base = base[~base.pol_insee_code.str[:2].isin(["2A","2B"])]
base["pol_insee_code"] = base["pol_insee_code"].astype(int)
base_nd =  base.drop_duplicates(subset=['pol_insee_code'])

input_data = input_data.drop_duplicates()
input_data = input_data[~input_data.pol_insee_code.str[:2].isin(["2A","2B"])]
input_data["pol_insee_code"] = input_data["pol_insee_code"].astype(int)

input_data = input_data.merge(base_nd, how='inner', on="pol_insee_code")
X0_f = input_data.iloc[:, 4:32]

X_f = (X0_f
      .drop(columns=['pol_coverage','pol_insee_code'])
      .fillna('Unknown').pipe(pd.get_dummies))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [134]:
G= G.iloc[:, :-5]

In [135]:
X_train, X_test, y_train, y_test, G_train, G_test, E_train, E_test, I_train, I_test  = train_test_split(X, y, G, E, I, test_size=0.2, random_state=7)

In [136]:
scaler = StandardScaler().fit(X_train)
scale_df = lambda df, scaler: pd.DataFrame(scaler.transform(df), columns=df.columns, index=df.index)
X_train = X_train.pipe(scale_df, scaler) 
X_test = X_test.pipe(scale_df, scaler)
X_f = X_f.pipe(scale_df, scaler)

In [137]:
scalerg = StandardScaler().fit(G_train)
scale_df_g = lambda df, scalerg: pd.DataFrame(scalerg.transform(df), columns=df.columns, index=df.index)
G_train = G_train.pipe(scale_df_g, scalerg) 
G_test = G_test.pipe(scale_df_g, scalerg)

In [138]:
y_traint =np.expand_dims(y_train,axis=1)
y_testt =np.expand_dims(y_test,axis=1)

### **GLM POISSON SANS ZONIER **

In [139]:
# ICI il n'y a que des expsures en 1 d'ou l'offset (log(expo)) non présent 
from sklearn import linear_model
clf = linear_model.PoissonRegressor() 
clf.fit(X_train,y_train)

PoissonRegressor()

In [140]:
def EDR_POIS(yhat, y):
  #loss=torch.mean(torch.exp(xbeta)-y*xbeta)
  #loss=torch.mean(yhat-y*torch.log(yhat))
  eps=0.000000000001
  res=1-np.mean((y*np.log((y+eps)/yhat)-(y-yhat)))/np.mean((y*np.log((y+eps)/np.mean(y))))
  return res

def gini_coefficient(x):
    """Compute Gini coefficient of array of values"""
    diffsum = 0
    for i, xi in enumerate(x[:-1], 1):
        diffsum += np.sum(np.abs(xi - x[i:]))
    return diffsum / (len(x)**2 * np.mean(x))

In [141]:
print(" GINI train : ", gini_coefficient(clf.predict(X_train)))
print(" GINI Poi test : ", gini_coefficient(clf.predict(X_test)))
print(" EDR POIS train : ", EDR_POIS(clf.predict(X_train),y_train))
print(" EDR POIS test : ", EDR_POIS(clf.predict(X_test),y_test))
MSE_Poi_tr =  ((clf.predict(X_train)-y_train)**2).mean()
MSE_Poi_test =  ((clf.predict(X_test)-y_test)**2).mean()
print(" MSE Poi train : ", MSE_Poi_tr)
print(" MSE Poi test : ", MSE_Poi_test)

 GINI train :  0.08660650208503035
 GINI Poi test :  0.08672764768037573
 EDR POIS train :  0.026278388992110213
 EDR POIS test :  0.02151244397947283
 MSE Poi train :  0.13392113526321742
 MSE Poi test :  0.13262211736790874


In [142]:
FREQ = clf.predict(X_train)
CM= reg.predict(X_train)

In [143]:
FREQ

array([0.11865199, 0.13119345, 0.12771253, ..., 0.11926551, 0.13316271,
       0.1625666 ])

In [144]:
CM

array([ 857.87415714, 1001.55870427, 1036.43410264, ..., 1382.45637353,
       1105.0085487 , 1159.50573026])

In [145]:
X_final=input_data

In [146]:
X_final['FREQ'] = clf.predict(X_f)
X_final['CM'] = reg.predict(X_f)
X_final['PP'] = X_final['FREQ']*X_final['CM']

In [147]:
X_final['PP'].mean()

132.52708196599963

In [148]:
input_data['claim_amount'].mean()

124.09248483493673

In [149]:
input_data

,id_client,id_vehicle,id_policy,id_year,pol_bonus,pol_coverage,pol_duration,pol_sit_duration,pol_pay_freq,pol_payd,pol_usage,pol_insee_code,drv_drv2,drv_age1,drv_age2,drv_sex1,drv_sex2,drv_age_lic1,drv_age_lic2,vh_age,vh_cyl,vh_din,vh_fuel,vh_make,vh_model,vh_sale_begin,vh_sale_end,vh_speed,vh_type,vh_value,vh_weight,vh_make_new,exposure,claim_nb,claim_amount,CODGEO,Nb Pharmacies et parfumerie,Dynamique Entrepreneuriale,Dynamique Entrepreneuriale Service et Commerce,Synergie Médicale COMMUNE,...,Moyenne Revnus fiscaux,Taux Evasion Client,"Nb Education, santé, action sociale",Nb Services personnels et domestiques,"Nb Santé, action sociale",Nb Industries des biens intermédiaires,Nb de Commerce,Nb de Services aux particuliers,"Nb institution de Education, santé, action sociale, administration",PIB Régionnal,SEG Environnement Démographique Obsolète,Score Croissance Population,Score Croissance Entrepreneuriale,Score VA Région,Score PIB,Environnement Démographique,Fidélité,SYN MEDICAL,Seg Cap Fiscale,Seg Dyn Entre,DYN SetC,CP,code_commune_INSEE,nom_commune_postal,code_postal,libelle_acheminement,ligne_5,latitude,longitude,code_commune,article,nom_commune,nom_commune_complet,code_departement,nom_departement,code_region,nom_region,FREQ,CM,PP
0,A00000001,V01,A00000001-V01,Year 0,0.5,Maxi,29,9,Biannual,No,Retired,36233,No,85,0,M,NaN,62,0,10,1587,98,Gasoline,PEUGEOT,306,10,9,182,Tourism,20700,1210,PEUGEOT,1,0,0.0,36233,0.0,41.0,19.0,57.0,...,10788.5,0.0,0.0,0.0,0.0,2916.0,3652.0,1226.0,6903.0,63457.0,Zone rurale en croissance démographique,69.26230,0.00173,12.30681,11.54726,Bassin Industriel en croissance démographique,Pop Sédentaire,Faible Synergie Médicale,Fiscalité moyenne,Faible dynamique,Faible Dynamique Serv et Com,36,36233,LA VERNELLE,36600,LA VERNELLE,NaN,47.242881,1.563677,233.0,La,Vernelle,La Vernelle,36,Indre,24.0,Centre-Val de Loire,0.110011,1131.664449,124.495267
1,A00019002,V01,A00019002-V01,Year 0,0.5,Median2,12,3,Yearly,No,WorkPrivate,36233,Yes,57,57,F,M,32,38,8,1868,70,Diesel,PEUGEOT,PARTNER,9,6,142,Tourism,14400,1185,PEUGEOT,1,0,0.0,36233,0.0,41.0,19.0,57.0,...,10788.5,0.0,0.0,0.0,0.0,2916.0,3652.0,1226.0,6903.0,63457.0,Zone rurale en croissance démographique,69.26230,0.00173,12.30681,11.54726,Bassin Industriel en croissance démographique,Pop Sédentaire,Faible Synergie Médicale,Fiscalité moyenne,Faible dynamique,Faible Dynamique Serv et Com,36,36233,LA VERNELLE,36600,LA VERNELLE,NaN,47.242881,1.563677,233.0,La,Vernelle,La Vernelle,36,Indre,24.0,Centre-Val de Loire,0.128139,932.144334,119.443991
2,A00000002,V01,A00000002-V01,Year 0,0.5,Maxi,3,1,Biannual,No,Retired,92073,No,69,0,M,NaN,39,0,4,2149,170,Diesel,MERCEDES BENZ,C220,4,2,229,Tourism,34250,1510,MERCEDES BENZ,1,0,0.0,92073,7.0,2368.0,3363.0,1333.0,...,10867.5,12.0,246.0,69.0,238.0,10383.0,28477.0,19139.0,39842.0,500839.0,Com > 200 m habts en croissance démographique,73.36066,0.94839,100.00000,100.00000,Bassin Urbain en croissance démographique,Pop Mobile,Forte Synergie Médicale,Fiscalité faible,Dynamique Economique,Bonne Dynamique Entreprise Serv et Com,92,92073,SURESNES,92150,SURESNES,NaN,48.869848,2.219655,73.0,NaN,Suresnes,Suresnes,92,Hauts-de-Seine,11.0,Île-de-France,0.156834,1539.875546,241.504162
3,A00007764,V01,A00007764-V01,Year 0,0.5,Maxi,2,2,Yearly,No,WorkPrivate,92073,Yes,40,38,M,F,22,17,6,2198,155,Diesel,JAGUAR,X-TYPE,6,1,215,Tourism,39350,1575,Other,1,0,0.0,92073,7.0,2368.0,3363.0,1333.0,...,10867.5,12.0,246.0,69.0,238.0,10383.0,28477.0,19139.0,39842.0,500839.0,Com > 200 m habts en croissance démographique,73.36066,0.94839,100.00000,100.00000,Bassin Urbain en croissance démographique,Pop Mobile,Forte Synergie Médicale,Fiscalité faible,Dynamique Economique,Bonne Dynamique Entreprise Serv et Com,92,92073,SURESNES,92150,SURESNES,NaN,48.869848,2.219655,73.0,NaN,Suresnes,Suresnes,92,Hauts-de-Seine,11.0,Île-de-France,0.138263,1161.157633,160.544962
4,A00011017,V01,A00011017-V01,Year 0,0.5,Maxi,13,1,Yearly,No,WorkPrivate,92073,No,69,0,M,NaN,45,0,5,1984,150,Ga

In [150]:
X_final.shape

(98417, 154)

In [151]:
input_data

,id_client,id_vehicle,id_policy,id_year,pol_bonus,pol_coverage,pol_duration,pol_sit_duration,pol_pay_freq,pol_payd,pol_usage,pol_insee_code,drv_drv2,drv_age1,drv_age2,drv_sex1,drv_sex2,drv_age_lic1,drv_age_lic2,vh_age,vh_cyl,vh_din,vh_fuel,vh_make,vh_model,vh_sale_begin,vh_sale_end,vh_speed,vh_type,vh_value,vh_weight,vh_make_new,exposure,claim_nb,claim_amount,CODGEO,Nb Pharmacies et parfumerie,Dynamique Entrepreneuriale,Dynamique Entrepreneuriale Service et Commerce,Synergie Médicale COMMUNE,...,Moyenne Revnus fiscaux,Taux Evasion Client,"Nb Education, santé, action sociale",Nb Services personnels et domestiques,"Nb Santé, action sociale",Nb Industries des biens intermédiaires,Nb de Commerce,Nb de Services aux particuliers,"Nb institution de Education, santé, action sociale, administration",PIB Régionnal,SEG Environnement Démographique Obsolète,Score Croissance Population,Score Croissance Entrepreneuriale,Score VA Région,Score PIB,Environnement Démographique,Fidélité,SYN MEDICAL,Seg Cap Fiscale,Seg Dyn Entre,DYN SetC,CP,code_commune_INSEE,nom_commune_postal,code_postal,libelle_acheminement,ligne_5,latitude,longitude,code_commune,article,nom_commune,nom_commune_complet,code_departement,nom_departement,code_region,nom_region,FREQ,CM,PP
0,A00000001,V01,A00000001-V01,Year 0,0.5,Maxi,29,9,Biannual,No,Retired,36233,No,85,0,M,NaN,62,0,10,1587,98,Gasoline,PEUGEOT,306,10,9,182,Tourism,20700,1210,PEUGEOT,1,0,0.0,36233,0.0,41.0,19.0,57.0,...,10788.5,0.0,0.0,0.0,0.0,2916.0,3652.0,1226.0,6903.0,63457.0,Zone rurale en croissance démographique,69.26230,0.00173,12.30681,11.54726,Bassin Industriel en croissance démographique,Pop Sédentaire,Faible Synergie Médicale,Fiscalité moyenne,Faible dynamique,Faible Dynamique Serv et Com,36,36233,LA VERNELLE,36600,LA VERNELLE,NaN,47.242881,1.563677,233.0,La,Vernelle,La Vernelle,36,Indre,24.0,Centre-Val de Loire,0.110011,1131.664449,124.495267
1,A00019002,V01,A00019002-V01,Year 0,0.5,Median2,12,3,Yearly,No,WorkPrivate,36233,Yes,57,57,F,M,32,38,8,1868,70,Diesel,PEUGEOT,PARTNER,9,6,142,Tourism,14400,1185,PEUGEOT,1,0,0.0,36233,0.0,41.0,19.0,57.0,...,10788.5,0.0,0.0,0.0,0.0,2916.0,3652.0,1226.0,6903.0,63457.0,Zone rurale en croissance démographique,69.26230,0.00173,12.30681,11.54726,Bassin Industriel en croissance démographique,Pop Sédentaire,Faible Synergie Médicale,Fiscalité moyenne,Faible dynamique,Faible Dynamique Serv et Com,36,36233,LA VERNELLE,36600,LA VERNELLE,NaN,47.242881,1.563677,233.0,La,Vernelle,La Vernelle,36,Indre,24.0,Centre-Val de Loire,0.128139,932.144334,119.443991
2,A00000002,V01,A00000002-V01,Year 0,0.5,Maxi,3,1,Biannual,No,Retired,92073,No,69,0,M,NaN,39,0,4,2149,170,Diesel,MERCEDES BENZ,C220,4,2,229,Tourism,34250,1510,MERCEDES BENZ,1,0,0.0,92073,7.0,2368.0,3363.0,1333.0,...,10867.5,12.0,246.0,69.0,238.0,10383.0,28477.0,19139.0,39842.0,500839.0,Com > 200 m habts en croissance démographique,73.36066,0.94839,100.00000,100.00000,Bassin Urbain en croissance démographique,Pop Mobile,Forte Synergie Médicale,Fiscalité faible,Dynamique Economique,Bonne Dynamique Entreprise Serv et Com,92,92073,SURESNES,92150,SURESNES,NaN,48.869848,2.219655,73.0,NaN,Suresnes,Suresnes,92,Hauts-de-Seine,11.0,Île-de-France,0.156834,1539.875546,241.504162
3,A00007764,V01,A00007764-V01,Year 0,0.5,Maxi,2,2,Yearly,No,WorkPrivate,92073,Yes,40,38,M,F,22,17,6,2198,155,Diesel,JAGUAR,X-TYPE,6,1,215,Tourism,39350,1575,Other,1,0,0.0,92073,7.0,2368.0,3363.0,1333.0,...,10867.5,12.0,246.0,69.0,238.0,10383.0,28477.0,19139.0,39842.0,500839.0,Com > 200 m habts en croissance démographique,73.36066,0.94839,100.00000,100.00000,Bassin Urbain en croissance démographique,Pop Mobile,Forte Synergie Médicale,Fiscalité faible,Dynamique Economique,Bonne Dynamique Entreprise Serv et Com,92,92073,SURESNES,92150,SURESNES,NaN,48.869848,2.219655,73.0,NaN,Suresnes,Suresnes,92,Hauts-de-Seine,11.0,Île-de-France,0.138263,1161.157633,160.544962
4,A00011017,V01,A00011017-V01,Year 0,0.5,Maxi,13,1,Yearly,No,WorkPrivate,92073,No,69,0,M,NaN,45,0,5,1984,150,Ga

In [156]:
X_final=input_data.iloc[:, 4:32]

In [157]:
X_final['FREQ'] = clf.predict(X_f)
X_final['CM'] = reg.predict(X_f)
X_final['PP'] = X_final['FREQ']*X_final['CM']

In [158]:
X_final.shape

(98417, 31)

In [159]:
X_final.to_csv('final.csv', header=True, index=None, sep=';')